<h1>Data Extraction</h1>

In [62]:
# imports
import pandas as pd
import csv
import openpyxl
import os
import json

import re

import requests
from bs4 import BeautifulSoup

from dotenv import load_dotenv

In [ ]:
# file names
SCHOOL_FED = "federal_college_data.csv"
SCHOOL_FORBES = "forbes_rankings.csv"
SCHOOL_RATINGS = "myplan_rankings.csv"
CITY_ZIPS = "us_cities_zip_county.csv"
CITY_ZIPS_JSON = "us_cities_zip_county.json"
CITY_CRIME = "us_cities_crime.csv"
# Zippopotam API used in the data_integration code to match zips to cities in the CITY_CRIME file

In [3]:
STATE_ABBREVS = {
    "alabama": "AL",
    "alaska": "AK",
    "arizona": "AZ",
    "arkansas": "AR",
    "california": "CA",
    "colorado": "CO",
    "connecticut": "CT",
    "delaware": "DE",
    "florida": "FL",
    "georgia": "GA",
    "hawaii": "HI",
    "idaho": "ID",
    "illinois": "IL",
    "indiana": "IN",
    "iowa": "IA",
    "kansas": "KS",
    "kentucky": "KY",
    "louisiana": "LA",
    "maine": "ME",
    "maryland": "MD",
    "massachusetts": "MA",
    "michigan": "MI",
    "minnesota": "MN",
    "mississippi": "MS",
    "missouri": "MO",
    "montana": "MT",
    "nebraska": "NE",
    "nevada": "NV",
    "new hampshire": "NH",
    "new jersey": "NJ",
    "new mexico": "NM",
    "new york": "NY",
    "north carolina": "NC",
    "north dakota": "ND",
    "ohio": "OH",
    "oklahoma": "OK",
    "oregon": "OR",
    "pennsylvania": "PA",
    "rhode island": "RI",
    "south carolina": "SC",
    "south dakota": "SD",
    "tennessee": "TN",
    "texas": "TX",
    "utah": "UT",
    "vermont": "VT",
    "virginia": "VA",
    "washington": "WA",
    "west virginia": "WV",
    "wisconsin": "WI",
    "wyoming": "WY",
    "district of columbia": "DC"
}

<h2>Validate CSV File Name</h2>

In [4]:
def validate_filename(filename: str) -> bool:

    # check for correct file extension
    if not filename.lower().endswith(".csv"):
        print("Error: Filename must end with .csv")
        return False

    # check for invalid characters
    invalid_characters = r'[<>:"/\\|?*\']'
    if re.search(invalid_characters, filename):
        print("Error: Filename contains invalid characters.")
        return False

    # check for empty or whitespace-only name
    if filename.strip() == ".csv":
        print("Error: Filename cannot be empty or just whitespace.")
        return False

    return True

<h2>Forbes Rankings Data</h2>
<h3>Variables of Interest</h3>
Institution Name, Rank, State, Average Grade, Median Base Salary, Student Population, Campus Setting, School Size, Description, Institution Type, Carnegie Classification, Student to Faculty Ratio, Total Grant Aid, Percent of Students Receive Financial Aid, Percent of Students Receive Grants

In [ ]:
def get_forbes_data(filename: str):
    # JSON API endpoint url from https://www.forbes.com/top-colleges/
    FORBES_URL = "https://www.forbes.com/forbesapi/org/top-colleges/2025/rank/true.json?fields=organizationName,academics,state,financialAid,rank,medianBaseSalary,campusSetting,studentPopulation,squareImage,uri,description,grade,schoolSize&limit=500&start=0"
    headers = {'User-Agent': 'Mozilla/5.0'}
    universities = []
    
    # validate filename to ensure that the data can be downloaded into a CSV file
    if not validate_filename(filename):
        return f"Invalid filename: {filename}. Please check your filename and try again."
    
    # call page and show error if unable to make the request
    try:
        response = requests.get(FORBES_URL, headers=headers)
        data = response.json()
    except requests.exceptions.RequestException as e: # helps diagnose issues with HTTP request
        print(f"Request error: {e}")
        return None
    except ValueError as e:
        print(f"JSON decoding error: {e}")
        return None
    else:
        try:
            schools = data['organizationList']['organizationsLists']
        except KeyError as e: # helps diagnose API structure changes
            print(f"Key error: {e}")
            print("Available keys in response:", data.keys())
            return None

    
    # loop through schools and collect data
    for school in schools:
        university = {
            'rank': school.get('rank'),
            'name': school.get('organizationName'),
            'state': school.get('state'),
            'grade': school.get('grade'),
            'medianBaseSalary': school.get('medianBaseSalary'),
            'studentPopulation': school.get('studentPopulation'),
            'campusSetting': school.get('campusSetting'),
            'schoolSize': school.get('schoolSize'),
            'description': school.get('description'),
            'uri': school.get('uri')
        }
    
        academics = school.get('academics', {})
        university['institutionType'] = academics.get('type')
        university['carnegieClassification'] = academics.get('carnegieClassification')
        university['studentFacultyRatio'] = academics.get('studentFacultyRatio')
    
        financial_aid = school.get('financialAid', {})
        university['totalGrantAid'] = financial_aid.get('totalGrantAid')
        university['percentOfStudentsFinAid'] = financial_aid.get('percentOfStudentsFinAid')
        university['percentOfStudentsGrant'] = financial_aid.get('percentOfStudentsGrant')
        
        # add all institution data to universities list, which can then be appended to a CSV file
        universities.append(university)
    
    # create dataframe from university data and extract into a CSV file
    df = pd.DataFrame(universities)
    df.to_csv(f"../data/{filename}", index=False)

    return f"{len(universities)} universities added to {filename}."

In [7]:
get_forbes_data(SCHOOL_FORBES)

'500 universities added to forbes_rankings.csv.'

<h2>myPlan Data</h2>
<h3>Variables of Interest</h3>
Institution Name, Prestige, Satisfaction, Resources & Facilities, Personal Safety, Teacher Support and Involvement, School Administration, Campus Setting, Aggregate score of all variables (Average Score)

In [14]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [35]:
def scrape_myplan_data(var_name, url_num, entries: int):
    # offset used to browse through page URLs, an offset of 400 lists schools ranked 401-500
    url = f"https://www.myplan.com/education/colleges/college_rankings_{url_num}.php?sort=1&offset="
    university_rankings = []
    
    for i in range(0, entries + 100, 100):
        page_url = url + str(i)

        # access url
        headers = {"User-Agent": "Mozilla/5.0"}  # Prevents blocking
        html = requests.get(page_url, headers=headers, verify=False).text
        soup = BeautifulSoup(html, "html.parser")

        # loacte part of the HTML structure that contains university data
        td_element = soup.find("td", {"background": "../../images/career_details_panel_bg_long.gif"})

        if td_element:
            rows = td_element.find_all("tr")
            for row in rows:
                cells = row.find_all("td")
                if len(cells) >= 3: 
                    name_tag = cells[1].find("a", class_="ratings_list")
                    score_tag = cells[2].find("div", align="right")
                    
                    if name_tag and score_tag:
                        name = name_tag.get_text(strip=True)
                        score = score_tag.get_text(strip=True)
                        try: #skip invalid scores
                            float(score)
                            university_rankings.append((name, score))
                        except ValueError:
                            continue
        else:
            print(f"No data found for offset {i}")
    
    print(f"{len(university_rankings)} total entries were found and documented for {var_name}.")
    return university_rankings


# merge data from all school variables into a csv sheet
# calculate avg score based on all variables
def get_myplan_data(data_source: list, filename: str):

    # scrape data
    headers, variables_list = [], []
    for var, url_num, entries in data_source:
        ranking_list = scrape_myplan_data(var, url_num, entries)
        headers.append(var)
        variables_list.append(ranking_list)
    

    # validate CSV file name
    if not validate_filename(filename):
        return f"Invalid filename: {filename}. Please check your filename and try again."

    num_variables = len(headers)
    merged = {}

    # make a list of scores for each school
    for var, ranking_list in enumerate(variables_list):
        for school, score in ranking_list:
            if school not in merged:
                merged[school] = [None] * num_variables # initialize as None to track missing data
            merged[school][var] = float(score)

    # construct final merged list with averages
    merged_list = []
    for school, scores in merged.items():
        non_none_scores = [s for s in scores if s is not None]
        avg_score = round(sum(non_none_scores) / len(non_none_scores), 2) if non_none_scores else None
        merged_list.append((school, *scores, avg_score))

    # ensure data folder exists
    os.makedirs("../data", exist_ok=True)

    with open(f"../data/{filename}", "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["school name", *headers, "average"])
        writer.writerows(merged_list)

    print(f"Saved to '../data/{filename}'")


In [37]:
# get myPlan ranking data from unique page per variable

variable_sources = [
    # category, url_num, #entries
    ("overall satisfaction", 1, 613),
    ("campus setting", 2, 613),
    ("student housing", 3, 610),
    ("resources", 4, 611),
    ("safety", 5, 612),
    ("teacher support", 6, 611),
    ("school admin", 7, 610),
    ("prestige", 8, 611),
    ("student competitiveness", 9, 612),
    ("student intelligence", 10, 612),
    ("party scene", 11, 506),
    ("greek life", 12, 505),
    ("student attractiveness", 13, 506 )
]

variable_sources1 = [
    # category, url_num, #entries
    ("overall satisfaction", 1, 613),
    ("campus setting", 2, 613)
]

get_myplan_data(variable_sources, SCHOOL_RATINGS)


613 total entries were found and documented for overall satisfaction.
613 total entries were found and documented for campus setting.
610 total entries were found and documented for student housing.
611 total entries were found and documented for resources.
612 total entries were found and documented for safety.
611 total entries were found and documented for teacher support.
610 total entries were found and documented for school admin.
611 total entries were found and documented for prestige.
612 total entries were found and documented for student competitiveness.
612 total entries were found and documented for student intelligence.
506 total entries were found and documented for party scene.
505 total entries were found and documented for greek life.
506 total entries were found and documented for student attractiveness.
Saved to '../data/myplan_rankings.csv'


<h2>Department of Education API</h2>

<h3>Variables Of Interest</h3>
Name - name - INSTNM (TEXT) <br>
State - school.state (TEXT) <br>
Admission rate - admission_rate.overall - ADM_RATE (FLOAT) <br>  
SAT Scores - admissions.sat_scores.average.overall <br>
Enrollment of all undergraduate students - enrollment.all - UG (INT)  <br> 
Average net price for Title IV institutions (public institutions) - avg_net_price.public - NPT4_PUB (INT) <br>  
Average net price for Title IV institutions (private for-profit and nonprofit institutions) - avg_net_price.private - NPT4_PRIV (INT)<br> 
Average cost of attendance (academic year institutions) - attendance.academic_year - COSTT4_A (INT)<br> 
The median debt for students who have completed - median_debt.completers.overall - GRAD_DEBT_MDN (FLOAT)<br> 
Median earnings of students working and not enrolled 8 years after entry - 8_yrs_after_entry.median_earnings - MD_EARN_WNE_P8 (FLOAT) <br>  
Mean earnings of students working and not enrolled 8 years after entry - 8_yrs_after_entry.mean_earnings - MN_EARN_WNE_P8 (FLOAT)  <br> 

In [ ]:
# example request
# https://api.data.gov/ed/collegescorecard/v1/schools?api_key=YOUR_API_KEY&fields=id,school.name,latest.cost.tuition.in_state,latest.completion.rate

# to request API key, visit: https://collegescorecard.ed.gov/data/api-documentation
# API default rate limit - 1,000 requests per IP address per hour
load_dotenv()  # Load from .env
secret_key = os.getenv("API_KEY")
API_KEY = secret_key
BASE_URL = "https://api.data.gov/ed/collegescorecard/v1/schools"

# variables of interest [add here to include additional variables in research]
# all variables can be found in ../data/CollegeScorecardDataDictionary.xlsx
FIELDS = [
    "school.zip",
    "school.name",
    "school.sector.scorecard",
    "school.city",
    "school.state",
    "latest.school.state_fips",
    "latest.school.locale",
    "latest.school.faculty_salary",
    "latest.admissions.sat_scores.average.overall",
    "latest.admissions.admission_rate.overall",
    "latest.completion.title_iv.completed_by.4yrs",
    "latest.cost.avg_net_price.public",
    "latest.cost.avg_net_price.private",
    "latest.cost.attendance.academic_year",
    "latest.aid.median_debt.completers.overall",
    "latest.earnings.8_yrs_after_entry.median_earnings",
    "latest.earnings.8_yrs_after_entry.mean_earnings",
    "latest.student.retention_rate.four_year.full_time_pooled",
    "latest.student.demographics.student_faculty_ratio"
]

HEADERS = [
    "Zip Code",
    "Name",
    "Type", # labeled in institution_types
    "City",
    "State",
    "State FIPS",
    "Locale",
    "Average Faculty Salary",
    "Average SAT Score",
    "Admission Rate",
    "4-Year Completion Rate",
    "Average Net Price (Public)",
    "Average Net Price (Private)",
    "Cost of Attendance (Academic Year)",
    "Median Debt of Completers",
    "Median Earnings (8 Years After Entry)",
    "Mean Earnings (8 Years After Entry)",
    "Retention Rate",
    "Student Faculty Ratio"
]

institution_types = {
    1: "Graduate-only Public institution",
    2: "Graduate-only Nonprofit institution",
    3: "Graduate-only For-profit institution",
    4: "4-year Public institution",
    5: "4-year Nonprofit institution",
    6: "4-year For-profit institution",
    7: "2-year Public institution",
    8: "2-year Nonprofit institution",
    9: "2-year For-profit institution",
    10: "Less-than-2-year Public institution",
    11: "Less-than-2-year Nonprofit institution",
    12: "Less-than-2-year For-profit institution",
    13: "Unknown Public institution",
    14: "Unknown Nonprofit institution",
    15: "Unknown For-profit institution"
}

def get_data_to_csv(start_page, end_page, filename):
    # validate CSV file name
    if not validate_filename(filename):
        return f"Invalid filename: {filename}. Please check your filename and try again."
    
    all_data = []
    page = start_page
    
    while page < end_page:
        params = {
            "api_key": API_KEY,
            "fields": ",".join(FIELDS),
            "per_page": 100,
            "page": page
        }

        response = requests.get(BASE_URL, params=params)

        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            print(response.text)
            break

        data = response.json()
        results = data.get("results", [])

        def safe_get(school, field):
            value = school.get(field)
            return value if value is not None else "N/A"
        
        for school in results:
            row = [safe_get(school, field) for field in FIELDS]
            all_data.append(row)

        page += 1
    
    # after fetching all data, save to CSV with human readable headers
    # there are thousands of institutions in this API, so we will extract data in portions to check for error
    # use append mode to add data at the end of the CSV file to prevent overwriting previous entries
    file_exists = os.path.isfile(f"../data/{filename}") # use os to prevent rewriting headers
    with open(f"../data/{filename}", "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists:
            writer.writerow(HEADERS)
        writer.writerows(all_data)
 
    print(f"Data from pages {start_page} to {end_page} saved to '../data/{filename}' - {len(all_data)} records")

In [ ]:
# 6429 school entries total as of 7/31/2025
get_data_to_csv(0, 66, SCHOOL_FED)

Data from pages 0 to 66 saved to '../data/federal_college_data.csv' - 6429 records


In [54]:
# check that all columns have at least some data

fed_df = pd.read_csv("../data/federal_college_data.csv", na_values=["N/A"])

print("Columns missing in every row:")
print(fed_df.columns[fed_df.isna().all()].tolist()) # none, meaning that all variables were extracted properly

Columns missing in every row:
[]


<h2>US City Population Data</h2>
US Cities by State and their population

In [60]:
def get_city_info(filename):
    # validate CSV file name
    # validate JSON filename
    if not filename.endswith(".json"):
        return f"Invalid filename: {filename}. Must end in '.json'."
    
    all_data = []

    for state, abbrev in STATE_ABBREVS.items():
        url = f"https://www.geonames.org/postal-codes/US/{abbrev}/{state}.html"

        try:
            response = requests.get(url)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch data for {state}: {e}")
            continue

        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table", {"class": "restable"})
        if not table:
            print(f"No data table found for {state}")
            continue

        rows = table.find_all("tr")[1:]  # Skip header row

        for row in rows:
            cells = row.find_all("td")
            if len(cells) >= 6:
                city = cells[1].get_text(strip=True)
                postal_code = cells[2].get_text(strip=True)
                state_full = cells[4].get_text(strip=True)
                county = cells[5].get_text(strip=True)

                all_data.append({
                    "City": city,
                    "Postal Code": postal_code,
                    "State": state_full,
                    "County": county,
                })

        output_path = f"../data/{filename}"
        try:
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(all_data, f, indent=2, ensure_ascii=False)
            return f"Data successfully written to '{output_path}'."
        except Exception as e:
            return f"Failed to write data to JSON: {e}"
    
    return f"Available data was added to '../data/{filename}'."

In [ ]:
# get data as json
get_city_info(CITY_ZIPS_JSON)

"Data successfully written to '../data/us_cities_zip_county.json'."

<h2>US Crime Data</h2>
Excel download available from the "Offenses Known To Law Enforcement” dataset by the Federal Bureau of Investigation (FBI) Crime Data ExplorerData - https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/home (By State and University)

In [ ]:
def expand_merged_cells(excel_file, sheet_name=0):
    wb = openpyxl.load_workbook(excel_file)
    sheet = wb[sheet_name] if isinstance(sheet_name, str) else wb.worksheets[sheet_name]

    # Expand merged cells (copy the value to all cells in the merged range)
    for merged_range in sheet.merged_cells.ranges:
        min_row, min_col, max_row, max_col = merged_range.bounds
        value = sheet.cell(row=min_row, column=min_col).value
        for row in range(min_row, max_row + 1):
            for col in range(min_col, max_col + 1):
                sheet.cell(row=row, column=col).value = value

    # Extract data into a list of lists
    data = []
    for row in sheet.iter_rows(values_only=True):
        data.append(list(row))

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)

    return df

# Example usage
df = expand_merged_cells("your_file.xlsx", sheet_name=0)
df.to_csv("flattened_output.csv", index=False, header=False)